In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import shutil
import matplotlib.pyplot as plt
import pylab as pl
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import History 
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import tensorflow.keras

import pandas as pd
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import itertools

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras import models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
train = '../input/train-images-enhanced'
val = '../input/val-test-images/val/val'
test = '../input/val-test-images/test'

In [ ]:
# Total number of training images
num_of_train_samples = 0
for train_dataset in os.listdir(train):
    in_folder = train + "/" + train_dataset 
    in_folder_list = os.listdir(in_folder)
    num_of_train_samples = num_of_train_samples + len(in_folder_list)
print("Number of training samples: ",num_of_train_samples)

# Total number of validation images
num_of_validation_samples = 0
for validation_dataset in os.listdir(val):
    in_folder_val = val + "/" + validation_dataset
    in_folder_val_list = os.listdir(in_folder_val)
    num_of_validation_samples = num_of_validation_samples + len(in_folder_val_list)
print("Number of validation samples: ", num_of_validation_samples)

num_of_test_samples = 0
for test_dataset in os.listdir(test):
    in_folder_test = test + "/" + test_dataset
    in_folder_test_list = os.listdir(in_folder_test)
    num_of_test_samples = num_of_test_samples + len(in_folder_test_list)
print("Number of test samples: ", num_of_test_samples)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=40,    
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
batch_size = 32
target_names = ['COVID', 'PNEUMONIA', 'NORMAL']

# Defining image width and height respectively
img_rows = 224
img_cols = 224

In [ ]:
train_generator = train_datagen.flow_from_directory(train,
                                                    target_size = (img_rows, img_cols),
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical')

validation_generator = val_datagen.flow_from_directory(val,
                                                        target_size = (img_rows, img_cols),
                                                        batch_size = batch_size,
                                                        shuffle = False, class_mode='categorical')

test_generator = val_datagen.flow_from_directory(test,
                                                target_size = (img_rows, img_cols),
                                                batch_size = batch_size,
                                                shuffle = False, class_mode='categorical')

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized Confusion Matrix")
    else:
        print('Unnormalized Confusion Matrix')
    
    import itertools
    print(cm)
    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Classe Real')
    plt.xlabel('Classe Predita')
    plt.tight_layout()

In [ ]:
steps_per_epoch = num_of_train_samples // batch_size
validation_steps = num_of_validation_samples // batch_size
epoch = 12
print("The incoming data for each epoch of the training set: ",steps_per_epoch)
print("Validate the data passed in for each epoch of the set: ",validation_steps)
learning_rate = 0.01

In [ ]:
from tensorflow.keras.applications import InceptionV3,DenseNet201,VGG19,ResNet50V2,MobileNetV2
resnet_model = Sequential()
resnet_model.add(ResNet50V2(include_top=False, 
                   pooling='max', 
                   weights='imagenet'))
resnet_model.add(Flatten())
resnet_model.add(layers.Dropout(0.4))
resnet_model.add(Dense(256, activation = "relu"))
resnet_model.add(BatchNormalization())
resnet_model.add(Dense(3, activation='softmax'))

In [ ]:
resnet_model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=learning_rate),
              metrics=["accuracy"])

In [ ]:
filepath="ResNet50V2.h5"
checkpoint = ModelCheckpoint(filepath=filepath,
        save_weights_only=False,
        monitor='val_accuracy',   
        mode='max',
        save_best_only=True)

resnet_training = resnet_model.fit(train_generator,
                               steps_per_epoch = steps_per_epoch,
                               epochs = epoch,
                               validation_data = validation_generator,
                               validation_steps = validation_steps,
                               callbacks=[checkpoint])

In [ ]:
plt.figure(figsize=(8,8),dpi=80)
plt.plot(resnet_training.history['accuracy'])
plt.plot(resnet_training.history['val_accuracy'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("epoch")
plt.legend(["train","test"],loc="lower right")
plt.show()

In [ ]:
plt.figure(figsize=(8,8),dpi=80)
plt.plot(resnet_training.history['loss'])
plt.plot(resnet_training.history['val_loss'])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train","test"],loc="lower right")
plt.show()

In [ ]:
Y_pred_res = resnet_model.predict_generator(test_generator, num_of_test_samples // batch_size+1)
y_pred_resnet = np.argmax(Y_pred_res, axis=1)
print('Confusion Matrix')
conf_matrix_res = confusion_matrix(test_generator.classes, y_pred_resnet)
cm_res = np.array2string(conf_matrix_res)
print(conf_matrix_res)
print("=============================================================================================")
print('Classification Report')
class_rep_res = classification_report(test_generator.classes, y_pred_resnet, target_names=target_names)
print(class_rep_res)